## 'Finding a Most Likely Common Ancestor'

**Connections**: `IPRB`, `SUBS`, `HAMM`

---


**Given**: A collection of at most 10 DNA strings of equal length (at most 1 kbp) in FASTA format.

**Return**: A consensus string and profile matrix for the collection. (If several possible consensus strings exist, then you may return any one of them.)


In [8]:
# Libraries to load

import numpy as np
import random


In [2]:
# Plan: load FASTA sequences, check lengths to make sure all equal, turn each into a list of characters, load into an array

bases = 'ACGT'

def fasta_parse_to_dict(path_to_filename):
    '''
    TEXT
    '''
    with open(path_to_filename, 'r') as f:
        lst  = f.readlines()
    f.close()
    for i in range(len(lst)):
        if lst[i].startswith('>'):
            lst[i] = lst[i].split(' ')[0]+'\n'
    lst  = [i.replace('\n', ' ') for i in lst]
    str1 = ''.join(lst)  
    lst2 = str1.split('>')
    lst2 = lst2[1:]
    dict1 = {lst2[i].split(' ')[0]:''.join(lst2[i].split(' ')[1:])
            for i in range(len(lst2))}
    del lst, lst2
    return dict1


def sequence_dict_to_array(sequence_dictionary):
    '''TEXT'''
    sequence_list = []
    key_list = list(sequence_dictionary.keys())
    for i in range(len(key_list)):
        sequence_list.append([k for k in sequence_dictionary[key_list[i]]] )
    try:
        seq_array = np.array(sequence_list)
        del sequence_list
        return seq_array
    except ValueError:
        print("Error: Sequence lengths are unequal.")


def profile_matrix(sequence_array):
    '''
    TEXT
    '''
    test_matrix = np.zeros((4,len(sequence_array[0])), np.int32)
    for i in range(len(sequence_array[0])):
        seq_slice = sequence_array[:, i]
        bases='ACGT'
        for j in range(len(bases)):
            test_matrix[j,i] = np.count_nonzero(seq_slice==bases[j])
        del seq_slice, bases
    return test_matrix


def consensus_sequence(profile_matrix):
    '''
    TEXT
    '''
    bases='ACGT'
    con_seq = ''
    std_matrix = profile_matrix/profile_matrix.sum(axis=0)
    for i in range(len(std_matrix[0])):
        x,y = 0,0
        for j in range(len(bases)):
            if std_matrix[j,i] > x:
                x = std_matrix[j,i]
                y=j
        con_seq += bases[y]
    return con_seq



In [3]:
#test_dict = fasta_parse_to_dict('datasets/rosalind_sample_dataset.txt')

test_profile = profile_matrix(sequence_dict_to_array(fasta_parse_to_dict('datasets/rosalind_sample_dataset.txt')))

test_profile

array([[5, 1, 0, 0, 5, 5, 0, 0],
       [0, 0, 1, 4, 2, 0, 6, 1],
       [1, 1, 6, 3, 0, 1, 0, 0],
       [1, 5, 0, 0, 0, 1, 1, 6]], dtype=int32)

In [7]:
print(consensus_sequence(test_profile))
for i in range(len(bases)):
    print(bases[i],':', ' '.join(map(str, test_profile[i])))

ATGCAACT
A : 5 1 0 0 5 5 0 0
C : 0 0 1 4 2 0 6 1
G : 1 1 6 3 0 1 0 0
T : 1 5 0 0 0 1 1 6


In [11]:
del test_profile

In [9]:
for x in range(10):
    print(''.join([random.choice(bases) for i in range(50)]))

AGTCGGGTGAGAATAGCTGTTTGATGTAAACGAAATCTAAATCATCTGCA
AATCCACGAGTTAGCCACAGCGCCTATGTAACTTTTTACTACGCCGCACC
ATTTCTAGCGCAGTTACATAAATCACTGCTGAAGACTCTCGCGGCTGAAC
ACCTGCCCCCTAGCCCGTGAGTAGCGTAGGTCGAAGTTCAGGTTAGGAAT
AATTGTTGCTAAGTGCCAGCGAAAGCGATCAAGCAGACCAGGAACTGATC
GAGGCAGCATTTTGGTTTACAGTCCGTACTTAACATCCGCTTGTCTCGCA
GGTGCTTAGTGATCTTAACTCATGCTACCTATCTCCCATTAGTAGATCGT
CACTTAAGACGTATGGCCCTGCGCCCTTGCGACTATCTACTGCACAATCA
CGGCCTAAATGATTGGGGGGACTAGTGCTAGCCACTCTCAGATAAGTGAA
GCGTCCACTGCAATGAGGGGACGTTGATCGAGTATCAATCTGGTGGTTTC


In [10]:
test_2 = profile_matrix(sequence_dict_to_array(fasta_parse_to_dict('datasets/rosalind_test_cons_dataset.txt')))
print(consensus_sequence(test_2))
for i in range(len(bases)):
    print(bases[i],':', ' '.join(map(str, test_2[i])))

del test_2

AATTCTAGATGAATGCCAGGAATCCGTACAAAAAATCTCAGGGACGTACA
A : 5 4 0 0 0 3 4 2 4 1 1 7 4 0 1 2 2 3 2 2 4 3 2 3 1 1 2 4 1 3 4 4 3 4 6 0 2 3 2 4 3 1 1 5 2 2 1 4 3 4
C : 2 2 2 3 6 2 2 3 3 2 2 0 0 2 2 3 4 2 2 2 2 3 1 4 4 3 0 2 4 2 1 3 3 2 2 3 5 3 4 4 0 2 2 1 5 1 2 1 4 4
G : 3 3 3 2 3 1 2 4 2 3 4 0 3 2 5 3 3 2 5 3 3 2 3 2 2 4 2 2 2 2 3 2 2 1 0 2 0 0 1 0 4 5 4 1 2 4 3 3 1 0
T : 0 1 5 5 1 4 2 1 1 4 3 3 3 6 2 2 1 3 1 3 1 2 4 1 3 2 6 2 3 3 2 1 2 3 2 5 3 4 3 2 3 2 3 3 1 3 4 2 2 2


---

### Problem Attempt

#### Attempt 1

In [14]:
problem = profile_matrix(sequence_dict_to_array(fasta_parse_to_dict('datasets/rosalind_cons.txt')))

print(consensus_sequence(problem))

for i in range(len(bases)):
    print(bases[i]+':', ' '.join(map(str, problem[i])))

del problem


ACCAGAGAGACCATAATCATAAAGCCCCGCGCGACTCTGTCTGAAGTAAGTTACCCATGACGCACTAAAGGCGAATAATATAAAGAAGTACGCCATACAGTGGCCAGTAGCTAAGCAGAAGAACCAAGCATCTCAAAATATGTGGAAAAGATGCAGAGAAAGCCCTAAGCGTCAGAAAGGTAAACGACCAGAAGAAGAGGAAGCACAAGAAAATTTCTGCCCGTAGGAGCATCCGCACGAAACAAAATATGGAATTATTGCCGAGACCTTGATATCCAGCAACAGATGCTCTCTCGACGGATAGATCTCAACTACTCCTCTAAAAGGCGCTTGACGACTAGACACTCTGACGGTTGATTCTTAACCCAACCAAGCCGAACACGAGGACTGTAGAAACAGGACGAAATCGCTACCACCAGTGGATGGCAGCCTTCACCGGGCTATCCCGTGCGTGCCCGTCCAGCGGTATATGTTAAGCTATGCCGCAGTTCGAAAGCGGGACTGGAACCCACTTCTCCATCGGGATTCCTTTGTCGAAACACTATGAGGGGCAGCAAGTCGAACATACATAAATGAGCTTTAAACCCGAAAGTGATCCGCTCACCAACGAATGAGAGAATAGAATGCATCAAAACCCGATGGATAGACAATCAAGCGAAACAAACGGCCACGTCCTTCCAACTAGTTTTGAGATAAGGCGGATTTCGGAGCGTAAGGGCAGACGGCCCGGTGCCACGGTCAATAAGATTAACGCCAATTACCGTACCGACATGTACCTGCGGACGCAAAACCAAAACGTATAGCGCAGATCGATACGAGGGCTAGCTAATACTCCAATACTCGCACGGCGTAGCACCAACCCATGACCGGCTCACAACGTTCACCTAGGTACTAAGCTTCCTCTCCAGCCATGAAACCACAGGACCAAGATG
A: 3 2 1 3 2 3 2 6 2 3 2 1 3 1 4 6 4 2 4 3 4 5 5 3 1 2 2 0 2 2 1 2 

In [13]:
fasta_parse_to_dict('datasets/rosalind_cons.txt')

{'Rosalind_2306': 'TATCGCTAACTCTAGATTGTGAAGGTGGATTACACTCTTTCTCTACGTACCTTTCAAGGGGGCTGCACACTCGAAAAATATCCTTAGGCTCACAACTGAGCGAGCAGTCGAGTATGTGACACATAAAGCAACTTCAATGGTGTCTGCGAGATTAGGGAGCGACACTACGTCACTTATTGGTAAGTTAAGAGAAATAGTGGACGATCATCCCCAACGCTTCCCGTGGGTGAACGCACTCGTTACATGATGGTGCAGTGTGTTGGATCCATCAATGTGTGGCTACTGGTGCGCTTTTTTAGGGCTGATACATGGTTACCGCATTAATGGCGGCAACGAAGGAGATGTTAAGACCCGGGTTTCAAAAGCGTCAAAATATGAATAAACGAGATGTCTACACTGCCCTCCAACTGAACACGCATTGGAGGGGCATGTGACCCGGCTAAGCATTCCCCTCATGACGTCGTAGTATTGTGATACGCAGCCCGCTATTAATTCCTGGCTTAAGGGACGAGGTCCTGGAGGTGTCTAAACAGTAATACCCTCCTGCGACCGAGTTAGTCCCCCTTGCTAAGTTTCTCGTAGTTAATGCTTTAGAAGAGCTGTGGCAGACGATCCCCAATAGTAAGATGCCGAAAACGAGGGACAGCGAATCAGCAGGAATTTACGTCCGTAGGTCTCCCATTATGTGCTAGTTCAGAGGGCGTGCAGACTATGAGGGCTGGCTGACGGGGTTCAGGGATCTTAGATTTAACCCATGCCTTATTAGGGGCCTGTTCCTGCGTGGATAATTCCGAATTAACTGCAATCGTCGGTTAGGAGGACCACGAAATACTATACTCGTCCCTGGCCGATCCATCACTGTCTGTACTTGGCATTATAATACATGATATTTACACCTCCCTCAGGACCGATGACGTGACCCTTACTAGATC',
 'Rosalind_4615': 'ACATGCGATAGCGTTGAAGCTGCAATT

#### Attempt 2

In [15]:
problem = profile_matrix(sequence_dict_to_array(fasta_parse_to_dict('datasets/rosalind_cons.txt')))

print(consensus_sequence(problem))

for i in range(len(bases)):
    print(bases[i]+':', ' '.join(map(str, problem[i])))

del problem

TAGCCCTCCACGGTGAAGAGAGCGCTATCACACTAGATAAACCAAACCAAGCAACATGAAAAAACGCCTCCGAAGACTCTACTAAATATGACGACAGTAGGCAAAGTGGCACTGCTCATACGAGATGATCATCCGCGTCAGCAAGCATAAAGGCACGGACGATCGAAGCCAACAATCAATCAGACGGCAAGAGACTCTTTTTCGTCGTAATTAAGAGCTCATCGCTAAGTTTGAAATAACATCTACAGTAGGACGCTACATCGAGTCGACCAACAGGTCAAAGATCTACCCGGCAAATAGAAGGAGAACATGTAAAGCAAACGAGGCCAGGCGGAGTTACGCTAAGTAATGGGTGAACCAACTCAATGGAGAACTATACCTTTGCACACTACCGCAGAACTCCAAGCATATACAGCGAGTTAATTCTAAACCGACCGAAACGTAACGCCTACACAAAAACAGGTCATGGATGACTCGCTGCGACAAGGCGACACTAAACCGCCCTTGGCTCCACGAGAGGCTCCCAACCTAAGTCGAACAAAACCACCGCTACTCAAGTCACACTGAGCCATATACCTAATATTGCTATGAGGGTCAGGAGTTACATATCCACGGAAGACGACACCGAGCGCTGAACCAACACCTCTCGATCATATTACCCGACATATGACGGAAAATACAAGATTAACTGCTTCCAATATGATTAGCAGAAACTACAAAGACTTGGAGACGTTCGGAATTTGGCAGATAGAGTTTAACCGAGGCGGTACAATAACATCCGACGCGCTGAAGGCGTTCCACAGCTAAACCCCAACACAACCTCCACTTGACATCAAACCGGCACAGGCAGACACTTGTTCTTCACAAAAGAAATCACCAATGACTAAACGTATGACCGCCCAACAAAGGAAACGAACGC
A: 3 3 2 2 2 0 2 1 4 5 3 2 1 1 2 4 3 1 3 3 4 1 2 1 1 2 3 1 2 4 1 3 1 2 3 2 3 3 5